In [1]:
import urllib.request

import geopandas
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import pandas as pd
import planetary_computer
import pystac_client
import rioxarray
import xarray as xr
import odc.stac
import dask.distributed

In [2]:
from shapely import wkt

In [3]:
# https://boundingbox.klokantech.com/

region = wkt.loads(
    "POLYGON((-58.611677 -34.469254, -58.271327 -34.469254, -58.271327 -34.725584, -58.611677 -34.725584, -58.611677 -34.469254))"
)
bbox = region.bounds

In [16]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1/",
    modifier=planetary_computer.sign_inplace,
)

collection = catalog.get_collection("ms-buildings")
asset = collection.assets["delta"]

storage_options = {
    "account_name": asset.extra_fields["table:storage_options"]["account_name"],
    "sas_token": asset.extra_fields["table:storage_options"]["credential"],
}

search = catalog.search(
    collections=["ms-buildings"],
    intersects=region,
    query={
        #"msbuildings:processing-date": {"gt": "2021-07-06"},
    },
)

items = search.item_collection()
len(items)

6

In [17]:
items

In [6]:
!pip install adlfs pyarrow


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [19]:
import adlfs

fs = adlfs.AzureBlobFileSystem(
    **items[0].assets["data"].extra_fields["table:storage_options"]
)

prefixes = [item.assets["data"].href for item in items]
parts = []
for item in items:
    parts.extend(fs.ls(item.assets["data"].href))

In [20]:
df = pd.concat(
    [
        geopandas.read_parquet(f"az://{part}", storage_options=storage_options)
        for part in parts
    ]
)
df.head()


KeyboardInterrupt



In [ ]:
belg = wkt.loads(
    'POLYGON((-58.478172 -34.551306, -58.440475 -34.551306, -58.440475 -34.567287, -58.478172 -34.567287, -58.478172 -34.551306))'
)


In [ ]:
belg_df = df[df.intersects(belg)]

In [ ]:
belg_df.to_csv('belg.csv')

In [ ]:
belg_df